In [1]:
import pandas as pd

## Ingestion

In [2]:
df = pd.read_csv("../data/incidents_train.csv", index_col=False, dtype=str)
df = df.rename(columns={"Unnamed: 0": "id", "hazard-category": "hazard_category", "product-category": "product_category"})

In [3]:
documents = df.to_dict(orient='records')

In [4]:
import minsearch

In [5]:
index = minsearch.Index(
    text_fields=['title', 'hazard_category', 'product_category', 'hazard', 'product'],
    keyword_fields=['id']
)

In [ ]:
index.fit(documents)

## RAG flow

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [9]:
prompt_template = """
You're a food hazard detection assistant. Answer the QUESTION based on the CONTEXT from the food-incident reports.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()


entry_template = """
 'title': {title}
 'hazard_category': {hazard_category}
 'product_category': {product_category}
 'hazard': {hazard}
 'product': {product}
""".strip()


def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [10]:
def llm(prompt, model='llama3.1'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    
    return response.choices[0].message.content

In [11]:
def rag(query, model='llama3.1'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model=model)
    return answer

In [ ]:
question = 'What are the products that contain listeria monocytogenes?'
answer = rag(question)
print(answer)

## Retrieval evaluation

In [13]:
df_question = pd.read_csv('../data/ground-truth-retrieval_300.tsv', delimiter="\t")

In [ ]:
df_question.head()

In [15]:
ground_truth = df_question.to_dict(orient='records')

In [ ]:
ground_truth[0]

In [17]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [18]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [19]:
from tqdm.auto import tqdm

In [20]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [int(d['id']) == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [ ]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

## Finding the best parameters

In [ ]:
df_question.describe()

In [82]:
df_validation = df_question[:100]
df_test = df_question[100:]

In [83]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [84]:
gt_val = df_validation.to_dict(orient='records')

In [85]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [86]:
param_ranges = {
    'title': (0.0, 3.0),
    'hazard_category': (0.0, 3.0),
    'product_category': (0.0, 3.0),
    'hazard': (0.0, 3.0),
    'product': (0.0, 3.0)
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [ ]:
simple_optimize(param_ranges, objective, n_iterations=20)

In [ ]:
def minsearch_improved(query):
    boost = {
        'title': 2.99,
        'hazard_category': 0.41,
        'product_category': 1.97,
        'hazard': 0.92,
        'product': 0.64
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

## RAG evaluation

In [89]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [ ]:
len(ground_truth)

In [ ]:
ground_truth[0]

In [93]:
record = ground_truth[0]
question = record['question']
answer_llm = rag(question)

In [ ]:
print(answer_llm)

In [ ]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

In [97]:
import json

In [98]:
df_sample = df_question.sample(n=200, random_state=1)

In [99]:
sample = df_sample.to_dict(orient='records')

In [ ]:
sample[0]

In [115]:
evaluations = {}

In [ ]:
for record in tqdm(sample):
    id = record['id']

    if id in evaluations:
        continue
    
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    print(evaluation)
    if "```" in evaluation:
        evaluation = evaluation.split("```")[1].lstrip("json")
    elif "\n\n" in evaluation:
        evaluation = evaluation.split("\n\n")[1]
    evaluation = json.loads(evaluation)

    evaluations[id] = {
        'id': id,
        'question': question,
        'answer_llm': answer_llm,
        'evaluation': evaluation
    }

In [ ]:
evaluations[1319]

In [195]:
#df_eval = pd.DataFrame(evaluations, columns=['id', 'question', 'relevance', 'answer', 'explanation'])
df_eval = pd.DataFrame(evaluations, columns=['id', 'question', 'relevance', 'explanation'])

In [196]:
for k,v in evaluations.items():
    new_entry = {
        'id': v['id'],
        'question': v['question'],
        'relevance': v['evaluation']['Relevance'],
        #'answer': v['answer_llm'],
        'explanation': v['evaluation']['Explanation']
    }
    df1 = pd.DataFrame(new_entry, index=[0])
    df_eval = pd.concat([df_eval, df1])

In [ ]:
df_eval.info()

In [132]:
# df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

# df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
# df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

# df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
# df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

# del df_eval['record']
# del df_eval['evaluation']

In [198]:
df_eval.reset_index(drop=True, inplace=True)

In [199]:
df_eval.to_csv('../data/rag-eval-llama3.1_96.tsv', sep="\t", index=False)

In [ ]:
df_eval.relevance.value_counts(normalize=True)

In [ ]:
df_eval[df_eval.relevance == 'NON_RELEVANT']